# 11. 항공 사진 내 선인장 식별
* 경진대회명
    * 항공 사진 내 선인장 식별 경진대회
* 미션
    * 항공 사진 내 선인장이 있을 확률 예측
* 문제유형
    * 이진분류
* 평가지표
    * ROC AUC
## 11.1 경진대회 이해
* 드론이 멕시코의 보호지역을 돌아다니며 찍은 항공 사진에서 선인장(cactus)을 찾아내는게 목표
* 데이터
    * train.zip : 훈련 이미지 데이터 압축 파일
    * test.zip : 테스트 이미지 데이터 압축 파일
    * train.csv : 훈련 이미지 데이터 파일명 및 타깃값(0 or 1)
    * sample_submission.csv : 샘플 제출 파일
## 11.2 탐색적 데이터 분석
### 11.2.1 데이터 둘러보기